# Using _egobox_ optimizer _Egor_

## Installation

In [ ]:
%pip install egobox

We import _egobox_ as _egx_ for short.

In [1]:
import numpy as np
import egobox as egx

You may setup the logging level to get optimization progress during the execution

In [2]:
# To display optimization information (none by default)
# import logging
# logging.basicConfig(level=logging.INFO)

## Example 1 : Continuous optimization

### Test functions

In [3]:
xspecs_xsinx = egx.to_specs([[0., 25.]])
n_cstr_xsinx = 0

def xsinx(x: np.ndarray) -> np.ndarray:
    x = np.atleast_2d(x)
    y = (x - 3.5) * np.sin((x - 3.5) / (np.pi))
    return y

In [4]:
xspecs_g24 = egx.to_specs([[0., 3.], [0., 4.]])
n_cstr_g24 = 2

# Objective
def G24(point):
    """
    Function g24
    1 global optimum y_opt = -5.5080 at x_opt =(2.3295, 3.1785)
    """
    p = np.atleast_2d(point)
    return - p[:, 0] - p[:, 1]

# Constraints < 0
def G24_c1(point):
    p = np.atleast_2d(point)
    return (- 2.0 * p[:, 0] ** 4.0
            + 8.0 * p[:, 0] ** 3.0 
            - 8.0 * p[:, 0] ** 2.0 
            + p[:, 1] - 2.0)

def G24_c2(point):
    p = np.atleast_2d(point)
    return (-4.0 * p[:, 0] ** 4.0
            + 32.0 * p[:, 0] ** 3.0
            - 88.0 * p[:, 0] ** 2.0
            + 96.0 * p[:, 0]
            + p[:, 1] - 36.0)

# Grouped evaluation
def g24(point):
    p = np.atleast_2d(point)
    return np.array([G24(p), G24_c1(p), G24_c2(p)]).T


### Continuous optimization with _Egor_

In [5]:
egor = egx.Egor(g24, xspecs_g24, 
                     n_doe=10, 
                     n_cstr=n_cstr_g24, 
                     cstr_tol=1e-3,
                     infill_strategy=egx.InfillStrategy.WB2,
                     target=-5.5,
                     # outdir="./out",
                     # hot_start=True
                    )  # see help(egor) for options

# Specify regression and/or correlation models used to build the surrogates of objective and constraints
#egor = egx.Egor(g24, xlimits_g24, n_cstr=n_cstr_g24, n_doe=10,
#                      regr_spec=egx.RegressionSpec.LINEAR,
#                      corr_spec=egx.CorrelationSpec.MATERN32 | egx.CorrelationSpec.MATERN52)  

In [6]:
res = egor.minimize(n_iter=30)
print(f"Optimization f={res.y_opt} at {res.x_opt}")
print("Optimization history: ")
print(f"Inputs = {res.x_hist}")
print(f"Outputs = {res.y_hist}")

Optimization f=[-5.50855850e+00 -1.90254906e-04  8.42145223e-04] at [2.32960044 3.17895806]
Optimization history: 
Inputs = [[2.50536221 1.52013849]
 [2.96405976 0.35886285]
 [1.02839185 2.85727816]
 [2.1269163  1.69476817]
 [0.13837811 3.6367697 ]
 [1.81433857 3.38774669]
 [1.54501977 2.27377221]
 [0.89790707 1.17312028]
 [1.22486759 2.68220248]
 [0.52118272 0.49110379]
 [2.4737711  3.32865092]
 [2.33122651 3.18430835]
 [2.32987233 3.17891783]
 [2.32960044 3.17895806]]
Outputs = [[-4.02550069e+00 -3.68595779e+00 -6.97632004e-01]
 [-3.32292261e+00 -1.79721009e+01  3.38931744e-01]
 [-3.88567001e+00 -1.13949875e+00  2.84474419e+00]
 [-3.82168446e+00 -4.50967403e-01 -2.17740770e+00]
 [-3.77514781e+00  1.50404623e+00 -2.06806751e+01]
 [-5.20208526e+00  1.16080722e+00 -3.41244757e-01]
 [-3.81879198e+00 -7.14514899e-01 -2.41579312e-01]
 [-2.07102735e+00 -2.78540513e+00  9.88892480e-01]
 [-3.90707007e+00 -1.12064952e+00  2.04485655e+00]
 [-1.01228651e+00 -2.69695803e+00 -5.14382794e+00]
 [-5.

## Example 2 : Mixed-integer optimization

### Test function

In [7]:
xspecs_mixint_xsinx = [egx.XSpec(egx.XType(egx.XType.INT), [0, 25])]
n_cstr_mixint_xsinx = 0

def mixint_xsinx(x: np.ndarray) -> np.ndarray:
    x = np.atleast_2d(x)
    if (np.abs(np.linalg.norm(np.floor(x))-np.linalg.norm(x))< 1e-8):
        y = (x - 3.5) * np.sin((x - 3.5) / (np.pi))
    else:
        raise ValueError(f"Bad input: mixint_xsinx accepts integer only, got {x}")
    return y

### Mixed-integer optimization with _Egor_

In [8]:
egor = egx.Egor(mixint_xsinx, xspecs_mixint_xsinx, 
                     n_doe=3, 
                     infill_strategy=egx.InfillStrategy.EI,
                     target=-15.12,
                    )  # see help(egor) for options
res = egor.minimize(n_iter=30)
print(f"Optimization f={res.y_opt} at {res.x_opt}")
print("Optimization history: ")
print(f"Inputs = {res.x_hist}")
print(f"Outputs = {res.y_hist}")

Optimization f=[-15.12161154] at [19.]
Optimization history: 
Inputs = [[11.]
 [ 8.]
 [20.]
 [24.]
 [19.]]
Outputs = [[  5.1356682 ]
 [  4.45696985]
 [-14.15453288]
 [  4.91604976]
 [-15.12161154]]


## Example 3 : More mixed-integer optimization

In the following example we see we can have other special integer type cases, where a component of x can take one value out of a list of ordered values (ORD type) or being like an enum value (ENUM type). Those types differ by the processing related to the continuous relaxation made behind the scene:
* For INT type, resulting float is rounded to the closest int value,
* For ORD type, resulting float is cast to closest value among the given valid ones,
* For ENUM type, one hot encoding is performed to give the resulting value.  

### Test function

In [9]:
# Objective function which takes [FLOAT, ENUM1, ENUM2, ORD] as input
# Note that ENUM values are passed as indice value eg either 0, 1 or 2 for a 3-sized enum  
def mixobj(X):
    # float
    x1 = X[:, 0]
    #  ENUM 1
    c1 = X[:, 1]
    x2 = c1 == 0
    x3 = c1 == 1
    x4 = c1 == 2
    #  ENUM 2
    c2 = X[:, 2]
    x5 = c2 == 0
    x6 = c2 == 1
    # int
    i = X[:, 3]

    y = (x2 + 2 * x3 + 3 * x4) * x5 * x1 + (x2 + 2 * x3 + 3 * x4) * x6 * 0.95 * x1 + i
    return y.reshape(-1, 1)

### Mixed-integer optimization with _Egor_

In [10]:
xtypes = [
    egx.XSpec(egx.XType(egx.XType.FLOAT), [-5.0, 5.0]),
    egx.XSpec(egx.XType(egx.XType.ENUM), tags=["blue", "red", "green"]),
    egx.XSpec(egx.XType(egx.XType.ENUM), xlimits=[2]),
    egx.XSpec(egx.XType(egx.XType.ORD), [0, 2, 3]),
]
egor = egx.Egor(mixobj, xtypes, seed=42)
res = egor.minimize(n_iter=10)
print(f"Optimization f={res.y_opt} at {res.x_opt}")
print("Optimization history: ")
print(f"Inputs = {res.x_hist}")
print(f"Outputs = {res.y_hist}")

Optimization f=[-14.25] at [-5.  2.  1.  0.]
Optimization history: 
Inputs = [[-1.90197486  2.          1.          3.        ]
 [ 1.36933896  1.          0.          2.        ]
 [-0.10843099  1.          0.          0.        ]
 [-4.73477511  0.          0.          3.        ]
 [ 3.11266243  2.          1.          2.        ]
 [ 0.33069418  2.          1.          0.        ]
 [ 4.47594664  2.          1.          0.        ]
 [-3.26619512  0.          0.          2.        ]
 [-2.14691496  2.          1.          0.        ]
 [-3.11946471  1.          0.          0.        ]
 [-2.72317583  0.          0.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]]
Outp

Note that `x_opt` result contains indices for corresponding optional tags list hence the second component should be read as 0="red", 1="green", 2="blue", while the third component was unamed 0 correspond to first enum value and 1 to the second one.

## Usage

In [12]:
help(egor)

Help on Egor in module builtins object:

class Egor(object)
 |  Egor(fun, n_cstr=0, cstr_tol=1e-06, n_start=20, n_doe=0, regression_spec=7, correlation_spec=15, infill_strategy=1, q_points=1, par_infill_strategy=1, infill_optimizer=1, n_clusters=1)
 |  
 |  Optimizer constructor
 |  
 |  fun: array[n, nx]) -> array[n, ny]
 |       the function to be minimized
 |       fun(x) = [obj(x), cstr_1(x), ... cstr_k(x)] where
 |          obj is the objective function [n, nx] -> [n, 1]
 |          cstr_i is the ith constraint function [n, nx] -> [n, 1]
 |          an k the number of constraints (n_cstr)
 |          hence ny = 1 (obj) + k (cstrs)
 |       cstr functions are expected be negative (<=0) at the optimum.
 |  
 |   n_cstr (int):
 |       the number of constraint functions.
 |  
 |   cstr_tol (float):
 |       tolerance on constraints violation (cstr < tol).
 |  
 |   xspecs (list(XSpec)) where XSpec(xtype=FLOAT|INT|ORD|ENUM, xlimits=[<f(xtype)>] or tags=[strings]):
 |       Specificati